# Phase 1 

### Preprocessing

In [1]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer

### Now load the pickle data into dataframes from the Output folder one directory above

In [2]:
def loadEverything():
    all_projects_bugreports = pd.read_pickle('Output/allBugReports.pickle')
    print("*** All Bug Reports are Loaded. ***")
    all_projects_source_codes = pd.read_pickle('Output/allSourceCodes.pickle')
    print("*** All Source Codes are Loaded. ***")
    return all_projects_bugreports, all_projects_source_codes

all_projects_bugreports, all_projects_source_codes = loadEverything()
display(all_projects_bugreports.iloc[0])
display(all_projects_bugreports.iloc[0].fix)
display(all_projects_bugreports.iloc[0].summary)
display(all_projects_bugreports.iloc[0].description)
# display(all_projects_source_codes.iloc[0].unprocessed_code)

all_projects_bugreports

*** All Bug Reports are Loaded. ***
*** All Source Codes are Loaded. ***


fix                  [org.apache.commons.collections.map.flat3map.j...
text                                                               NaN
fixdate                                            2006-07-18 22:02:11
summary              Flat3Map.Entry.setValue() overwrites other Ent...
description          Flat3Map&amp;apos;s Entry objects will overwri...
project                                                    COLLECTIONS
average_precision                                                    0
Name: 217, dtype: object

array(['org.apache.commons.collections.map.flat3map.java',
       'org.apache.commons.collections.map.testflat3map.java'],
      dtype='<U52')

'Flat3Map.Entry.setValue() overwrites other Entry values'

'Flat3Map&amp;apos;s Entry objects will overwrite other Entry&amp;apos;s values if Entry.setValue() is called on one.  It should only overwrite the Entry at hand.\nI&amp;apos;ve looked at the source, and the case statement incorrectly falls through, rather than returning like it should:\nFlat3Map.java, lines 646-660:\n        public Object setValue(Object value) {\n            if (canRemove == false) \n{\n                throw new IllegalStateException(AbstractHashedMap.SETVALUE_INVALID);\n            }\n            Object old = getValue();\n            switch (nextIndex) \n{\n                case 3: \n                    parent.value3 = value;\n                case 2:\n                    parent.value2 = value;\n                case 1:\n                    parent.value1 = value;\n            }\n            return old;\n        }\nWith this code, if I set the value of the third item in the EntrySet, then all three values are set to the new value.'

,fix,text,fixdate,summary,description,project,average_precision
id,,,,,,,
217,[org.apache.commons.collections.map.flat3map.j...,NaN,2006-07-18 22:02:11,Flat3Map.Entry.setValue() overwrites other Ent...,Flat3Map&amp;apos;s Entry objects will overwri...,COLLECTIONS,0.0
214,[org.apache.commons.collections.testextendedpr...,NaN,2006-07-18 22:44:33,ExtendedProperties - field include should be n...,"The field ""include"" in ExtendedProperties is c...",COLLECTIONS,0.0
222,[org.apache.commons.collections.testlistutils....,NaN,2006-08-18 19:01:22,CollectionUtils removeAll is actually retainAll,"The removeAll(Collection collection, Collectio...",COLLECTIONS,0.0
261,[org.apache.commons.collections.map.flat3map.j...,NaN,2007-08-20 14:11:54,Flat3Map.remove() does not return the correct ...,final Flat3Map m = new Flat3Map();\n ...,COLLECTIONS,0.0
264,[org.apache.commons.collections.fasttreemap.java],NaN,2007-08-31 09:39:59,FastTreeMap forgets the comparator,In line 359 and 582 of the current 3.2 release...,COLLECTIONS,0.0
...,...,...,...,...,...,...,...
692,[org.wildfly.security.auth.realm.legacypropert...,NaN,2016-11-02 09:35:48,Add tests for special chars in LegacyPropertie...,Add tests for issue https://issues.jboss.org/b...,ELY,0.0
691,[org.wildfly.security.auth.realm.legacypropert...,NaN,2016-11-02 09:36:13,Elytron properties-realm is not compatible wit...,When users properties file (e.g. mgmt-users.pr...,ELY,0.0
637,[org.wildfly.security.auth.server.serverauthen...,NaN,2016-11-03 15:03:29,No log messages comming from Elytron - permiss...,Elytron is missing any log messages related to...,ELY,0.0


In [3]:

display(all_projects_bugreports.iloc[0].fix)
display(all_projects_source_codes.filename)


array(['org.apache.commons.collections.map.flat3map.java',
       'org.apache.commons.collections.map.testflat3map.java'],
      dtype='<U52')

0     \gitrepo\src\java\org\apache\commons\collectio...
1     \gitrepo\src\java\org\apache\commons\collectio...
2     \gitrepo\src\java\org\apache\commons\collectio...
3     \gitrepo\src\java\org\apache\commons\collectio...
4     \gitrepo\src\java\org\apache\commons\collectio...
                            ...                        
63    \gitrepo\src\main\java\org\wildfly\security\ut...
64    \gitrepo\src\main\java\org\wildfly\security\_p...
65    \gitrepo\src\test\java\org\wildfly\security\ma...
66    \gitrepo\src\test\java\org\wildfly\security\ma...
67    \gitrepo\src\test\java\org\wildfly\security\ss...
Name: filename, Length: 10461, dtype: object

## Removing composite varaibles

In [4]:
#remove next line characters:
def remove_new_lines(x):
    return str(x).replace('\n', '').replace('*', '').replace('/', '').replace('\t','')

def clean_new_lines_source_code(df):
    df.unprocessed_code = df.unprocessed_code.apply(remove_new_lines)
    return df

def clean_new_lines_bug_report(df):
    df.summary = df.summary.apply(remove_new_lines)
    df['description'] = df['description'].astype('|S')
    df.description = df.description.apply(remove_new_lines)
    return df

def format_filename(x):
  
    x = x.split('apache')
    if len(x) != 2:
        x = x[0].split('springframework')
        if len(x) != 2:
            x = x[0].split('wildfly')
            if len(x) == 2:
                x = 'org.wildfly' + x[1]
        else:
             x = 'org.springframework' + x[1]
    else:
         x = 'org.apache' + x[1]
    if len(x) == 1:
        x = x[0]
   
    x = str(x).replace("\\",".")
    return x

def clean_filepath(df):
    df.filename = df.filename.apply(format_filename)
    return df



In [5]:
import re

#splitting composite words
def findCompositeWords(s):
    return ' '.join(re.findall('[A-Z][^A-Z]*', s))   


def clean_composite_source_code(df):
    df.unprocessed_code = df.unprocessed_code.apply(findCompositeWords)
    return df

def clean_composite_bug_report(df):
    df.summary = df.summary.apply(findCompositeWords)
    df.description = df.description.apply(findCompositeWords)
    return df


### put Kelvin's key word cleaning above this

In [6]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
# add the Java key words to the stop words

# TODO we need to add the operators in here too
java_keywords = ["abstract", "assert**","assert", "boolean", "break", "byte", "case", "catch", "char", "const", "continue", "default", "do", "double", "else", "enum", "enum****" "extends", "final", "finally", "for", "goto","goto*", "if", "implements", "import", "instanceof", "int","interface", "long", "native", "new", "package", "private", "protected", "public", "return", "short", "static", "strictfp**","strictfp", "super", "switch", "synchornized", "this", "throw", "throws", "transient", "try", "void", "volatile", "while"]
java_operators = ["+", "-", "*", "/", "%", "+=", "-=", "*=", "/=", "++", "--", "==", "!=", "<", ">", "<=", ">=", ".", "[", "]", "(",")", "!", "~","instanceof", "<<", ">>", ">>>", "&", "^", "|", "&&", "||", "?", ":", "^=", "%=", "<<=", ">>=", ">>>=", "&="]
stop = java_keywords + java_operators
STOP_WORDS = ENGLISH_STOP_WORDS.union(stop)
# STOP_WORDS


#vect = CountVectorizer(min_df = 5, stop_words = stop, analyzer = 'word')

### Start working on TF-IDF and Cosine similarity calculations

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# clean up the unprocessed code column
def clean_source_df(sc_df):
    sc_df = clean_new_lines_source_code(sc_df)
    sc_df = clean_composite_source_code(sc_df)
    sc_df = clean_filepath(sc_df)
    return sc_df

def clean_bug_df(sc_df):
    sc_df = clean_new_lines_bug_report(sc_df)
    sc_df = clean_composite_bug_report(sc_df)
    return sc_df

# get clean versions of the dataframes
sc_df = clean_source_df(all_projects_source_codes)
br_df = clean_bug_df(all_projects_bugreports)

# TODO figure out which we need from the bug report 
#           do we use summary or description or both?
# bug_df = clean_source_code_list(all_projects_bugreports)


# get a list of the projects and their parent project
print(sc_df.project.unique())
projects = sc_df.project.unique()


# group the data frames
grouped_df = sc_df.groupby(sc_df.project)

# example of getting a single data frame
col_df = grouped_df.get_group("COLLECTIONS")
display(col_df)

# Won't use any of this
# commons_projects = projects[0:4]
# spring_projects = projects[4:11]
# wildfly_projects = projects[11]

# config_df = grouped_df.get_group(commons_projects[1])
# ely_df = grouped_df.get_group(wildfly_projects)
# ldap_df = grouped_df.get_group(spring_projects[3])

# print(config_df.iloc[0].filename)
# print(col_df.iloc[0].filename)
# print(ely_df.iloc[0].filename)
# print(ldap_df.iloc[0].filename)


['COLLECTIONS' 'CONFIGURATION' 'IO' 'LANG' 'DATACMNS' 'DATAMONGO'
 'DATAREST' 'LDAP' 'SEC' 'SOCIALFB' 'SPR' 'ELY']


,filename,unprocessed_code,project
0,org.apache.commons.collections.arraystack.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
1,org.apache.commons.collections.bag.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
2,org.apache.commons.collections.bagutils.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
3,org.apache.commons.collections.beanmap.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
4,org.apache.commons.collections.bidimap.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
...,...,...,...
471,org.apache.commons.collections.set.testtransfo...,Licensed to the Apache Software Foundation ...,COLLECTIONS
472,org.apache.commons.collections.set.testtypedse...,Licensed to the Apache Software Foundation ...,COLLECTIONS
473,org.apache.commons.collections.set.testtypedso...,Licensed to the Apache Software Foundation ...,COLLECTIONS
474,org.apache.commons.collections.set.testunmodif...,Licensed to the Apache Software Foundation ...,COLLECTIONS


In [8]:

def gen_parent_df(group, names):
    result = pd.DataFrame
    temp = []
    for name in names:
        temp.append(group.get_group(name))
    
    result = pd.concate(temp, ignore_index = true)    
    return result

# don't know if we are looking at these groups OR each of the projects individually
# commons_df = gen_parent_df(grouped_df, commons_projects)
# sprint_df = gen_parent_df(grouped_df, spring_projects)
# wildfly_df = gen_parent_df(grouped_df, wildfly_projects)




In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


def calculate_similarity(source_code, bug):
    cosSim = cosine_similarity(bug, source_code).flatten()
    return cosSim

# fit the vectorizer and transform data
def transform_data(source_code_data, query_data):
    # DO WE SET min_df?  default normalization is l2
    s = [source_code_data]
    q = [query_data]
    vect = TfidfVectorizer(min_df=1).fit(s)
    X = vect.transform(s)
    y = vect.transform(q)
    
    
    similarity = calculate_similarity(X,y)
     
    return X, y, similarity

# remove the stem and stop words
def stem_stop(text):
    stemmer = PorterStemmer()   #"english"
    text = text.split()
    text = [w for w in text if not w in STOP_WORDS]
    text = list(map(lambda x: stemmer.stem(x), text))
    text = ' '.join(text)
    text = text.strip()

    return text

def findBug(bug, source_codes):#  think of better name, take a bug report, and all projects source codes
    scores = {}

    for index, row in source_codes.iterrows():
        source_code = stem_stop(row['unprocessed_code'])
        bug_report = stem_stop(np.array_str(bug))
        
        if source_code != 0 and bug_report!= 0:
            try:
                similarity = transform_data(source_code, bug_report)
                scores[row['filename']]= similarity[2]
                print(similarity[2])
            except:
                print("hello")
    return scores

        
    
# first stem and remove stop words from the data frames column
# example 

# bug_df['unprocessed_code'] = bug_df['unprocessed_code'].map(stem_stop)

# then get the dataframe column you want to pass as source_code_data and as the query_data(bug)
bug = br_df.at['217','summary']

source = sc_df.at[0,'unprocessed_code']


scores = findBug(bug,sc_df)

print(scores)


[0.2579347]
[0.40279579]
[0.10565737]
[0.31396538]
[0.3952648]
[0.14493575]
[0.20412415]
[0.11607205]
[0.38556783]
[0.22052714]
[0.22494178]
[0.21153926]
[0.05591962]
[0.27977627]
[0.12732653]
[0.16424887]
[0.12151911]
[0.24081639]
[0.29064996]
[0.47466637]
[0.16638871]
[0.34815531]
[0.16780311]
[0.27796928]
[0.26556107]
[0.31823608]
[0.36167816]
[0.3400193]
[0.24257967]
[0.29134282]
[0.47209321]
[0.14939952]
[0.22656301]
[0.38087541]
[0.21688366]
[0.47647059]
[0.28566731]
[0.33981964]
[0.28377355]
[0.48099012]
[0.23112508]
[0.32849734]
[0.36113581]
[0.31575062]
[0.084119]
[0.26795754]
[0.48549103]
[0.36186654]
[0.21320072]
[0.30151134]
[0.38032866]
[0.37106878]
[0.35450672]
[0.52595168]
[0.32561911]
[0.09066482]
[0.22026087]
[0.11938872]
[0.38515101]
[0.07615857]
[0.31200998]
[0.18385094]
[0.34448841]
[0.28312766]
[0.36514837]
[0.10916311]
[0.19633468]
[0.20809753]
[0.21432896]
[0.19559056]
[0.26095236]
[0.41805667]
[0.1947033]
[0.29289108]
[0.23393819]
[0.28127795]
[0.40498038]
[0.38

[0.25978702]
[0.2083367]
[0.1345726]
[0.20073762]
[0.16190219]
[0.2425946]
[0.27136021]
[0.17763472]
[0.15583272]
[0.13130499]
[0.18021955]
[0.38591473]
[0.40245847]
[0.13968597]
[0.27648777]
[0.23372535]
[0.229047]
[0.08312476]
[0.12291987]
[0.2555548]
[0.14514375]
[0.21826084]
[0.1075749]
[0.12608907]
[0.31900677]
[0.33163177]
[0.23819653]
[0.30826285]
[0.14235505]
[0.10364294]
[0.13810521]
[0.1818671]
[0.16289259]
[0.28638262]
[0.31463405]
[0.14568897]
[0.27685403]
[0.31426968]
[0.38640491]
[0.35920458]
[0.31220495]
[0.31427278]
[0.31195539]
[0.3110199]
[0.22975187]
[0.06784656]
[0.06157929]
[0.08336244]
[0.09669011]
[0.1346198]
[0.12920916]
[0.11632561]
[0.05230658]
[0.25167066]
[0.09476226]
[0.08119828]
[0.04335314]
[0.13899857]
[0.09772234]
[0.22157426]
[0.16623034]
[0.15743261]
[0.12928598]
[0.35878209]
[0.30652763]
[0.28942284]
[0.05137155]
[0.36496126]
[0.16317321]
[0.13570677]
[0.35995289]
[0.10367288]
[0.20230071]
[0.20526716]
[0.4623641]
[0.32822109]
[0.60255693]
[0.6685663

[0.30382515]
[0.17002516]
[0.1381083]
[0.31622777]
[0.27586863]
[0.18319204]
[0.28554359]
[0.14625448]
[0.28287892]
[0.17039349]
[0.308301]
[0.29888848]
[0.34034946]
[0.26843775]
[0.25691]
[0.28325391]
[0.40824829]
[0.15372581]
[0.18056108]
[0.15912389]
[0.240617]
[0.3276236]
[0.]
[0.12281216]
[0.41420545]
[0.30313729]
[0.14957578]
[0.21378616]
[0.24886325]
[0.2826116]
[0.35355339]
[0.49815837]
[0.21564596]
[0.35566664]
[0.33536487]
[0.45161355]
[0.3712727]
[0.32530002]
[0.26524748]
[0.32899802]
[0.42640143]
[0.20826247]
[0.22473329]
[0.15425439]
[0.15430335]
[0.2080534]
[0.15358133]
[0.]
[0.16093253]
[0.06118936]
[0.07539069]
[0.]
[0.07136791]
[0.15249857]
[0.21744483]
[0.14693128]
[0.28194703]
[0.33049951]
[0.2469267]
[0.2849882]
[0.22621878]
[0.15290806]
[0.]
[0.13218777]
[0.23110035]
[0.20783549]
[0.33164583]
[0.15396007]
[0.13834414]
[0.17383782]
[0.09087999]
[0.12680459]
[0.34024395]
[0.28205677]
[0.19092174]
[0.37977509]
[0.]
[0.29008147]
[0.31090274]
[0.26613294]
[0.21450119]
[

[0.23845524]
[0.24003968]
[0.24498947]
[0.24084383]
[0.23924353]
[0.23845524]
[0.24003968]
[0.28490144]
[0.24500466]
[0.24419656]
[0.18450624]
[0.27961639]
[0.23787197]
[0.27665837]
[0.16731151]
[0.23522227]
[0.17952981]
[0.33323726]
[0.12200843]
[0.21320072]
[0.21821789]
[0.11952286]
[0.11952286]
[0.22086305]
[0.32267745]
[0.21081851]
[0.21081851]
[0.11952286]
[0.11785113]
[0.22645541]
[0.14285714]
[0.17213259]
[0.38575837]
[0.22407824]
[0.17263854]
[0.20254787]
[0.31231618]
[0.27126775]
[0.16726504]
[0.35728427]
[0.23735633]
[0.18172434]
[0.23795399]
[0.14519487]
[0.23112508]
[0.19428016]
[0.23945423]
[0.28420449]
[0.34014173]
[0.16171156]
[0.18325417]
[0.22005942]
[0.21765182]
[0.20432378]
[0.21199958]
[0.28794705]
[0.07123199]
[0.3254936]
[0.13821565]
[0.13052734]
[0.16647802]
[0.10220782]
[0.19733426]
[0.24294669]
[0.09600307]
[0.22825413]
[0.34294834]
[0.18359702]
[0.41275041]
[0.48951872]
[0.24572751]
[0.12361347]
[0.24177284]
[0.13245483]
[0.25475472]
[0.21402364]
[0.2125691]
[

[0.39864275]
[0.58625086]
[0.39877814]
[0.45325961]
[0.54638999]
[0.55787021]
[0.60216623]
[0.60961466]
[0.41985956]
[0.22124245]
[0.22105763]
[0.16464391]
[0.20109335]
[0.20073126]
[0.23190841]
[0.12077707]
[0.03944013]
[0.02759792]
[0.03766718]
[0.1718108]
[0.0541451]
[0.12107015]
[0.11012261]
[0.13895045]
[0.12233555]
[0.03945851]
[0.02760564]
[0.03769168]
[0.12309149]
[0.17269415]
[0.04776386]
[0.1242118]
[0.1929197]
[0.11972868]
[0.30512693]
[0.10316787]
[0.34740417]
[0.18898224]
[0.22668803]
[0.36402881]
[0.14664264]
[0.16271459]
[0.25372173]
[0.13881933]
[0.2432256]
[0.18444198]
[0.08412574]
[0.06840616]
[0.05219899]
[0.11648072]
[0.07722065]
[0.06816312]
[0.07681155]
[0.10310785]
[0.08595899]
[0.13730876]
[0.08648237]
[0.11210059]
[0.17872664]
[0.07548194]
[0.05686416]
[0.07047706]
[0.19538365]
[0.1946401]
[0.28942722]
[0.06296349]
[0.33183679]
[0.18017057]
[0.2422407]
[0.22397731]
[0.32668447]
[0.25427604]
[0.1204207]
[0.25471335]
[0.19703964]
[0.21386921]
[0.24504901]
[0.2988

...

[0.19278508]
[0.06561787]
[0.11785113]
[0.27272727]
[0.14657903]
[0.13084684]
[0.20355497]
[0.14135157]
[0.11331178]
[0.12751534]
[0.]
[0.21744428]
[0.31279938]
[0.18749043]
[0.2551276]
[0.34166266]
[0.39186884]
[0.19566844]
[0.23693401]
[0.19838491]
[0.29631214]
[0.2565783]
[0.22823241]
[0.32316107]
[0.20024163]
[0.19691682]
[0.22941573]
[0.16754156]
[0.33094818]
[0.31473566]
[0.1916068]
[0.20378479]
[0.22404847]
[0.31263295]
[0.260725]
[0.21760748]
[0.25782627]
[0.20327394]
[0.33872196]
[0.18826321]
[0.27959812]
[0.1383756]
[0.13749775]
[0.14027355]
[0.17971015]
[0.20412415]
[0.34142289]
[0.24829226]
[0.24462572]
[0.31877224]
[0.31595438]
[0.24378321]
[0.31153346]
[0.18534294]
[0.18569746]
[0.09669084]
[0.07283957]
[0.15111118]
[0.23550209]
[0.09515721]
[0.12215378]
[0.11074868]
[0.14526699]
[0.20119782]
[0.17489493]
[0.]
[0.1620915]
[0.14316407]
[0.20570448]
[0.21713012]
[0.18803383]
[0.21073145]
[0.09518415]
[0.11945462]
[0.11145564]
[0.15598534]
[0.15040726]
[0.25053788]
[0.217526

[0.13991375]
[0.10329178]
[0.28867513]
[0.23590713]
[0.08019245]
[0.09520212]
[0.19763781]
[0.23184732]
[0.19771421]
[0.26072901]
[0.13867505]
[0.17389124]
[0.17689267]
[0.1191377]
[0.1694515]
[0.09926739]
[0.12441946]
[0.13634214]
[0.17728379]
[0.1506933]
[0.12787805]
[0.07520855]
[0.26849685]
[0.28552638]
[0.27066426]
[0.19403763]
[0.19885812]
[0.23893829]
[0.2024972]
[0.26118658]
[0.16815711]
[0.14071951]
[0.23014365]
[0.04227659]
[0.15046628]
[0.1729765]
[0.1700973]
[0.17488113]
[0.19526616]
[0.17244389]
[0.12233382]
[0.10160624]
[0.03975254]
[0.12219464]
[0.13301503]
[0.08481718]
[0.08782441]
[0.11952286]
[0.07042283]
[0.09316543]
[0.0338635]
[0.11459314]
[0.09936608]
[0.09690075]
[0.07406874]
[0.10797479]
[0.10699395]
[0.11385103]
[0.21604621]
[0.24137414]
[0.25819889]
[0.11576969]
[0.39903815]
[0.18344333]
[0.22519541]
[0.08876952]
[0.16991242]
[0.13772648]
[0.14679517]
[0.10564483]
[0.07696863]
[0.11867817]
[0.1314955]
[0.12038332]
[0.10557002]
[0.15257798]
[0.20953952]
[0.0588

[0.12611406]
[0.08654198]
[0.09186785]
[0.202355]
[0.16705549]
[0.22243591]
[0.21512489]
[0.20137403]
[0.17935548]
[0.17638687]
[0.12129457]
[0.12670245]
[0.17818591]
[0.13085598]
[0.11684125]
[0.14784425]
[0.14285714]
[0.23027771]
[0.11593472]
[0.20123354]
[0.12871441]
[0.12209354]
[0.14026657]
[0.18718038]
[0.17524991]
[0.26393244]
[0.36170362]
[0.25461201]
[0.2845372]
[0.25458754]
[0.19931712]
[0.20953952]
[0.23547016]
[0.23354968]
[0.15151515]
[0.24830834]
[0.21940519]
[0.27338837]
[0.24854126]
[0.19745748]
[0.23773307]
[0.2731155]
[0.27056689]
[0.19351927]
[0.24198467]
[0.186339]
[0.27801922]
[0.28381844]
[0.26767233]
[0.17439503]
[0.09039502]
[0.23122575]
[0.06654944]
[0.11611695]
[0.08882848]
[0.19581511]
[0.10687334]
[0.13566469]
[0.11047364]
[0.20371195]
[0.15476465]
[0.13608276]
[0.18843367]
[0.21342821]
[0.23594513]
[0.19611614]
[0.13216372]
[0.13576885]
[0.21342821]
[0.30512858]
[0.20511412]
[0.16162441]
[0.21118547]
[0.13483997]
[0.2372895]
[0.17609018]
[0.16516803]
[0.199

[0.21192339]
[0.37783092]
[0.25819889]
[0.22513156]
[0.06112394]
[0.07090812]
[0.26726124]
[0.10882144]
[0.11785113]
[0.22281245]
[0.25884855]
[0.07699905]
[0.07968191]
[0.18006581]
[0.22155636]
[0.18705753]
[0.11878517]
[0.17154656]
[0.20301542]
[0.14363436]
[0.203664]
[0.19557232]
[0.2236068]
[0.22032632]
[0.19552949]
[0.17891501]
[0.23224434]
[0.18060946]
[0.18129928]
[0.10680171]
[0.16087173]
[0.41478068]
[0.11103184]
[0.08369002]
[0.13247244]
[0.10025608]
[0.09170315]
[0.10235307]
[0.06799322]
[0.]
[0.21374669]
[0.2003084]
[0.06312072]
[0.09935327]
[0.07744246]
[0.19672237]
[0.07980869]
[0.13858397]
[0.10323358]
[0.09494054]
[0.09514375]
[0.]
[0.0825991]
[0.10693485]
[0.0765122]
[0.18032125]
[0.15705699]
[0.14335102]
[0.10703249]
[0.13646894]
[0.13048116]
[0.14558648]
[0.08674361]
[0.15750057]
[0.1034329]
[0.16870468]
[0.07295174]
[0.18770961]
[0.08742604]
[0.10365164]
[0.]
[0.11839468]
[0.12870605]
[0.1304373]
[0.07511203]
[0.12539478]
[0.10808443]
[0.07403049]
[0.22963468]
[0.09

[0.17251347]
[0.11487594]
[0.18198447]
[0.18425061]
[0.12408128]
[0.25171509]
[0.18061488]
[0.21107926]
[0.1889026]
[0.26142569]
[0.15566744]
[0.13628513]
[0.13307513]
[0.16975674]
[0.16907496]
[0.20459437]
[0.15001649]
[0.23570226]
[0.19973802]
[0.1454004]
[0.19656021]
[0.28284271]
[0.15975241]
[0.2024578]
[0.12100478]
[0.248452]
[0.15430335]
[0.13012001]
[0.20321715]
[0.12194038]
[0.20954953]
[0.15075567]
[0.2754685]
[0.18989285]
[0.49590352]
[0.22351446]
[0.22852287]
[0.29740876]
[0.22808578]
[0.27271408]
[0.22742941]
[0.15811388]
[0.21758837]
[0.23747582]
[0.36514837]
[0.045511]
[0.15430335]
[0.22404699]
[0.14285714]
[0.08769204]
[0.19428197]
[0.17952588]
[0.09658293]
[0.21918683]
[0.15811388]
[0.14028612]
[0.17231138]
[0.15371167]
[0.17288756]
[0.14594537]
[0.12262899]
[0.21035717]
[0.21986264]
[0.24232223]
[0.36514837]
[0.26726124]
[0.17157765]
[0.247994]
[0.39530298]
[0.14285714]
[0.12384435]
[0.11762162]
[0.24376905]
[0.08980265]
[0.04737411]
[0.10448459]
[0.10163945]
[0.027213

In [ ]:
# similarity vector should be a data frame with bug_report, bug_report_fix, source_code.filename, and the similarity score

# another function that determines if the source_code file name is in the bugreport fix array. 



In [ ]:
print(type(scores['org.apache.commons.collections.arraystack.java']))